In [4]:
# rag_core.py
import sys; print("PY:", sys.executable)

import os, json
from typing import List, Dict, Optional
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery, VectorizedQuery
from azure.core.credentials import AzureKeyCredential

from azure.storage.blob import BlobServiceClient

from openai import AzureOpenAI

load_dotenv()

# ---------- Config ----------
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY = os.getenv("AZURE_SEARCH_API_KEY")
USE_IV = os.getenv("USE_INTEGRATED_VECTORIZATION", "true").lower() == "true"

VECTOR_FIELD = os.getenv("VECTOR_FIELD", "text_vector")
TEXT_FIELD   = os.getenv("TEXT_FIELD", "chunk")

AOAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AOAI_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AOAI_CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]

BLOB_CONNECTION_STRING = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
BLOB_CONTAINER = os.environ["AZURE_BLOB_CONTAINER"]

PY: /Users/felipesilverio/Documents/GitHub/azure_langproject/.venv/bin/python


In [6]:
# --- Cell 1: Azure AI Search smoke test ---
import os
from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError

load_dotenv(find_dotenv(), override=True)

SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # optional if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD", "text_vector")
TEXT_FIELD      = os.getenv("TEXT_FIELD", "chunk")

cred = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
sc = SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=cred)

query_text = "What are the company names with annual report"
print("Search endpoint:", SEARCH_ENDPOINT)
print("Index:", SEARCH_INDEX)
print("Vector field:", VECTOR_FIELD, "| Text field:", TEXT_FIELD)

try:
    # Try integrated vectorization first
    vq = VectorizableTextQuery(text=query_text, k=5, fields=VECTOR_FIELD)
    results = sc.search(search_text=None, vector_queries=[vq], top=5)
    mode = "vector (integrated)"
except HttpResponseError as e:
    print("Vector query failed, falling back to lexical search:", repr(e))
    results = sc.search(search_text=query_text, top=5)  # lexical fallback
    mode = "lexical"

print(f"\nMode: {mode}")
n = 0
for r in results:
    n += 1
    doc = dict(r)
    print(f"- score={r['@search.score']:.4f} | title={doc.get('title')} | chunk_id={doc.get('chunk_id')}")
    print((doc.get(TEXT_FIELD) or "")[:200], "...\n")

print("Total results returned:", n)


k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


Search endpoint: https://teneosearchpaid.search.windows.net
Index: rag-companyhouse1
Vector field: text_vector | Text field: chunk

Mode: vector (integrated)
- score=0.6631 | title=accounts-with-accounts-type-group_AA_2019-07-08_6.pdf | chunk_id=acd4d28b5b90_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvdHJ5b3V0LzExNDg4MTY2L2FjY291bnRzLXdpdGgtYWNjb3VudHMtdHlwZS1ncm91cF9BQV8yMDE5LTA3LTA4XzYucGRm0_pages_416
of free company reports These are typically from overseas-based 'brokers' who target UK shareholders offering to sell them what olten turn out to be worthless or high-risk shares in US or UK investmen ...

- score=0.6617 | title=objectprojection.json | chunk_id=595e828be1e0_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvdHJ5b3V0L2FIUjBjSE02THk5aGFYQnliMnBsWTNSMFpXNWxieTVpYkc5aUxtTnZjbVV1ZDJsdVpHOTNjeTV1WlhRdmRISjViM1YwTDJGSVVqQmpTRTAyVEhrNWFHRllRbmxpTW5Cc1dUTlNNRnBYTld4aWVUVnBZa2M1YVV4dFRuWmpiVlYxWkRKc2RWcEhPVE5qZVRWMVdsaFJkbVJJU2pWaU0xWXdURE5TYkdNelVYbE1ia0

In [16]:
# --- Cell 2: Azure OpenAI chat-completions smoke test ---
import os
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI, APIConnectionError
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv(), override=True)

endpoint    = os.environ["AZURE_OPENAI_ENDPOINT"]           # https://<resource>.openai.azure.com
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21")
deployment  = os.environ["AZURE_OPENAI_DEPLOYMENT"]         # your deployment name (e.g., gpt-4o-mini / o3-mini)
api_key     = os.getenv("AZURE_OPENAI_API_KEY")             # optional if using AAD

client = (
    AzureOpenAI(azure_endpoint=endpoint, api_key=api_key, api_version=api_version)
    if api_key
    else AzureOpenAI(azure_endpoint=endpoint, azure_ad_token_provider=DefaultAzureCredential().get_token, api_version=api_version)
)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Say 'pong' if you can read this."},
]

print("Endpoint:", endpoint)
print("Deployment:", deployment)
print("API version:", api_version)

# Non-streaming (most reliable across networks)
resp = client.chat.completions.create(model=deployment, messages=messages)
print("Non-streaming reply:", resp.choices[0].message.content)

# Optional: Streaming check (may fail behind some proxies / PE DNS)
try:
    text = ""
    stream = client.chat.completions.create(
        model=deployment,
        messages=messages,
        stream=True,
        # optional: get a final usage event
        stream_options={"include_usage": True},
    )
    for chunk in stream:
        # Some events (e.g., content filter/usage) have no choices—skip safely
        choices = getattr(chunk, "choices", None)
        if not choices:
            continue
        delta = getattr(choices[0], "delta", None)
        if not delta:
            continue
        piece = getattr(delta, "content", None)
        if piece:
            text += piece
            # print(piece, end="", flush=True)  # uncomment to see tokens live
    answer = text if text else "(no text returned)"
except APIConnectionError:
    # Non-streaming fallback (often succeeds behind strict proxies/PEs)
    resp = client.chat.completions.create(
        model=deployment,
        messages=messages,
    )
    answer = resp.choices[0].message.content

print("Streaming reply:", answer)


Endpoint: https://teneoproject2.openai.azure.com/
Deployment: gpt-5
API version: 2024-12-01-preview
Non-streaming reply: pong
Streaming reply: pong


In [1]:
# --- Cell A: wire Search + Azure OpenAI into a tiny RAG helper ---

import os, textwrap
from typing import List, Dict, Optional

from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError

from openai import AzureOpenAI, APIConnectionError

load_dotenv(find_dotenv(), override=True)

# ---- Config (expects the same envs you already used) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

AOAI_ENDPOINT   = os.environ["AZURE_OPENAI_ENDPOINT"]            # https://<resource>.openai.azure.com
AOAI_API_VER    = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21")
AOAI_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]          # e.g., gpt-4o-mini / o3-mini / gpt-5 preview
AOAI_KEY        = os.getenv("AZURE_OPENAI_API_KEY")              # omit if using AAD

# ---- Clients ----
def get_search_client() -> SearchClient:
    cred = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
    return SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=cred)

def get_aoai_client() -> AzureOpenAI:
    if AOAI_KEY:
        return AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, api_key=AOAI_KEY, api_version=AOAI_API_VER)
    return AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, azure_ad_token_provider=DefaultAzureCredential().get_token, api_version=AOAI_API_VER)

# ---- Retrieval (integrated vectorization first, lexical fallback) ----
def retrieve(query: str, k: int = 10):
    sc = get_search_client()
    try:
        vq = VectorizableTextQuery(text=query, k=k, fields=VECTOR_FIELD)
        # Prefer vector-only search (integrated vectorization). If your index isn't set up for it, this raises.
        results = sc.search(search_text=None, vector_queries=[vq], top=k)
        mode = "vector (integrated)"
    except HttpResponseError as e:
        # Fall back to lexical so you still get results while fixing vector config
        results = sc.search(search_text=query, top=k)
        mode = f"lexical (fallback due to: {e.__class__.__name__})"

    hits: List[Dict] = []
    for r in results:
        d = dict(r)
        d["score"] = r["@search.score"]
        hits.append(d)
    return mode, hits

def build_context(hits: List[Dict], text_field: str = TEXT_FIELD, max_chars: int = 4000) -> str:
    """Build a compact, numbered context block to feed the model."""
    lines = []
    total = 0
    for i, h in enumerate(hits, 1):
        title     = h.get("title")
        chunk_id  = h.get("chunk_id")
        snippet   = (h.get(text_field) or "")
        if not snippet:
            continue
        snippet = textwrap.shorten(snippet, width=700, placeholder=" ...")
        block = f"[{i}] title={title!r} | chunk_id={chunk_id} | score={h.get('score'):.4f}\n{snippet}"
        if total + len(block) > max_chars:
            break
        total += len(block)
        lines.append(block)
    return "\n\n---\n\n".join(lines)

# ---- RAG ask with streaming + non-streaming fallback ----
def rag_answer(question: str, k: int = 5, temperature: float = 0.2):
    mode, hits = retrieve(question, k=k)
    ctx = build_context(hits)

    system_msg = (
        "You are a financial analyst. Use ONLY the provided context to answer. "
        "All the files that you will be working with and PROVIDED in the context are annual reports. The name of the company that own the annual report is in the first page."
        "Cite sources using [#] that match the snippet numbers. "
        "If the answer isn't in the context, say you don't know."
    )
    user_msg = f"Question:\n{question}\n\nContext snippets (numbered):\n{ctx}"

    client = get_aoai_client()
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user",   "content": user_msg},
    ]

    # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.
    try:
        text = ""
        stream = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            stream=True,
            stream_options={"include_usage": True},
        )
        for chunk in stream:
            choices = getattr(chunk, "choices", None)
            if not choices:
                continue
            delta = getattr(choices[0], "delta", None)
            if not delta:
                continue
            piece = getattr(delta, "content", None)
            if piece:
                text += piece
        answer = text if text else "(no text returned)"
        mode_model = "streaming"
    except APIConnectionError:
        resp = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
        )
        answer = resp.choices[0].message.content
        mode_model = "non-streaming (fallback)"

    return {
        "search_mode": mode,
        "model_mode": mode_model,
        "answer": answer,
        "sources": [
            {"n": i+1, "title": h.get("title"), "chunk_id": h.get("chunk_id"), "score": h.get("score")}
            for i, h in enumerate(hits)
        ],
    }


In [4]:
from prompts import new_system_finance_prompt

system_msg = (
            new_system_finance_prompt
        )

print(system_msg)


You are a restructuring analyst focused on identifying companies in financial distress that could be advisory targets for your company. You prepare comprehensive, accurate and full one-pager profiles highlighting liquidity issues, debt maturity risks and covenant pressure. You rely on annual reports and financial statements of companies.

Each profile includes the following sections, with the following content and sourcing logic:

1. Introduction Table (Company Snapshot):

- This section provides a brief snapshot of the company. Include in a table format the following information of the target company, using the latest available annual report/financial statement of the company: 
-- Primary Industry (1–2-word label, e.g. automotive, gold mining, travel etc.)
-- Incorporation Year (official incorporation/founding date of the company)
-- Headquarters (city, country)
-- Number of Employees
-- Operational KPIs (These can vary for each company depending on what they report, but they cannot 

In [2]:
from collections import OrderedDict

def list_unique_titles(limit=5000):
    sc = get_search_client()
    print("Index:", getattr(sc, "index_name", "(unknown)"))

    results = sc.search(
        search_text="*",
        top=limit,
        select=["title"],              # make sure `title` is retrievable in your index
        include_total_count=True,
    )
    titles = OrderedDict()
    for d in results:
        t = d.get("title")
        if t:
            titles.setdefault(t, 0)
            titles[t] += 1

    print("Total docs:", results.get_count())
    print("Unique titles:", len(titles))
    for i, (t, n) in enumerate(titles.items(), 1):
        print(f"{i:>3}. {t}  (docs:{n})")

list_unique_titles()

/Users/felipesilverio/Documents/GitHub/azure_langproject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Index: (unknown)
Total docs: 80
Unique titles: 1
  1. seaport_topco.pdf  (docs:80)


In [2]:
# --- Cell B: run a RAG query and show results ---

question = "Give me a full list of company names with annual reportings?"
res = rag_answer(question, k=100, temperature=0.2)

print(f"Search mode: {res['search_mode']} | Model mode: {res['model_mode']}\n")
print("Answer:\n", res["answer"], "\n")
print("Sources:")
for s in res["sources"]:
    print(f"  [{s['n']}] title={s['title']!r} | chunk_id={s['chunk_id']} | score={s['score']:.4f}")


/Users/felipesilverio/Documents/GitHub/azure_langproject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


Search mode: vector (integrated) | Model mode: streaming

Answer:
 - Seaport Topco Limited [1] 

Sources:
  [1] title='seaport_topco.pdf' | chunk_id=db69abca9a6b_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvY29tcGFuaWVzaG91c2VzaW5nbGVmaWxlLzE0MTcxOTYyL3NlYXBvcnRfdG9wY28ucGRm0_pages_0 | score=0.6039
  [2] title='seaport_topco.pdf' | chunk_id=db69abca9a6b_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvY29tcGFuaWVzaG91c2VzaW5nbGVmaWxlLzE0MTcxOTYyL3NlYXBvcnRfdG9wY28ucGRm0_pages_61 | score=0.6017
  [3] title='seaport_topco.pdf' | chunk_id=db69abca9a6b_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvY29tcGFuaWVzaG91c2VzaW5nbGVmaWxlLzE0MTcxOTYyL3NlYXBvcnRfdG9wY28ucGRm0_pages_64 | score=0.5944
  [4] title='seaport_topco.pdf' | chunk_id=db69abca9a6b_aHR0cHM6Ly9haXByb2plY3R0ZW5lby5ibG9iLmNvcmUud2luZG93cy5uZXQvY29tcGFuaWVzaG91c2VzaW5nbGVmaWxlLzE0MTcxOTYyL3NlYXBvcnRfdG9wY28ucGRm0_pages_2 | score=0.5883
  [5] title='seaport_topco.pdf' | chunk_id=db69abca9a6b_

In [8]:
import os, textwrap
import io
from typing import List, Dict, Optional
from xml.sax.saxutils import escape

from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError
from azure.search.documents.models import HybridSearch

# 🔁 OpenAI (standard) SDK — Responses API
from openai import OpenAI, APIConnectionError

from prompts import finance_prompt_web

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

load_dotenv(find_dotenv(), override=True)

# ---- Config (same Azure Search envs you already use) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

# ---- OpenAI (standard) config ----
OPENAI_API_KEY  = os.getenv("FELIPE_OPENAI_API_KEY")        # required
OPENAI_MODEL    = os.getenv("FELIPE_OPENAI_MODEL", "gpt-5")  # e.g., "gpt-5" or "gpt-5-mini"

# ------------------ CODE

class profileAgentWeb():
    """
    Hybrid (dense+sparse) RAG over Azure AI Search.
    Generates Company Profiles with OpenAI GPT-5 (Responses API).
    Activated by 'Create company profile'.
    """

    def __init__(
        self,
        company_name: str,
        k: int,
        max_text_recall_size: int,
        max_chars: int,
        model: Optional[str] = None,
        *,
        # 🧠 GPT-5 tunables (defaults are sensible; override per call if you like)
        # temperature: float = 0.2,
        # top_p: float = 1.0,
        max_output_tokens: int = 1200,
        reasoning_effort: str = "medium",      # "minimal" | "low" | "medium" | "high"
        verbosity: str = "medium",                 # "low" | "medium" | "high"
        enable_web_search: bool = True,        # keep OFF by default to honor RAG "use only context"
        tool_choice: str = "auto",              # "none" | "auto" | {"type":"tool","name":"..."}
        streaming: bool = False,
        profile_prompt = finance_prompt_web
    ):
        self.company_name = company_name
        self.k = k
        self.max_text_recall_size = max_text_recall_size
        self.max_chars = max_chars

        # LLM settings
        self.model = model or OPENAI_MODEL
        # self.temperature = temperature
        # self.top_p = top_p
        self.max_output_tokens = max_output_tokens
        self.reasoning_effort = reasoning_effort
        self.verbosity = verbosity
        self.enable_web_search = enable_web_search
        self.tool_choice = tool_choice
        self.streaming = streaming

        self.profile_prompt = profile_prompt

        # Azure Search (unchanged)
        self.azure_credentials = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
        self.search_client = SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=self.azure_credentials)

        # OpenAI standard client
        self.web_openai = OpenAI(api_key=OPENAI_API_KEY)

    def _retrieve_hybrid_enhanced(self, query: str, k: int = 10, max_text_recall_size:int = 200):
        sc = self.search_client
        try:
            vq = VectorizableTextQuery(text=query, k=k, fields=VECTOR_FIELD)
            results = sc.search(
                search_text=query,
                vector_queries=[vq],
                top=self.k,
                query_type="semantic",
                query_caption="extractive",
                hybrid_search=HybridSearch(max_text_recall_size=self.max_text_recall_size),
                query_caption_highlight_enabled=True,
            )
            mode = "hybrid + semantic"
        except HttpResponseError as e:
            results = sc.search(search_text=query, top=k)
            mode = f"lexical (fallback due to: {e.__class__.__name__})"

        hits: List[Dict] = []
        for r in results:
            d = r.copy() if hasattr(r, "copy") else {k2: r[k2] for k2 in r}
            d["score"] = d.get("@search.reranker_score") or d.get("@search.score") or 0.0
            caps = d.get("@search.captions")
            if isinstance(caps, list) and caps:
                d["caption"] = getattr(caps[0], "text", None)
            hits.append(d)
        return mode, hits

    def _build_context(self, hits: List[Dict], text_field: str = TEXT_FIELD, max_chars: int = 20000) -> str:
        """Build a compact, numbered context block to feed the model."""
        lines = []
        total = 0
        for i, h in enumerate(hits, 1):
            title     = h.get("title")
            chunk_id  = h.get("chunk_id")
            snippet   = (h.get(text_field) or "")
            if not snippet:
                continue
            snippet = textwrap.shorten(snippet, width=700, placeholder=" ...")
            block = f"[{i}] title={title!r} | chunk_id={chunk_id} | score={h.get('score'):.4f}\n{snippet}"
            if total + len(block) > self.max_chars:
                break
            total += len(block)
            lines.append(block)
        return "\n\n---\n\n".join(lines)

    def _generate_pdf(self, text: str) -> bytes:
        buf = io.BytesIO()
        doc = SimpleDocTemplate(buf, pagesize=letter)
        styles = getSampleStyleSheet()
        body = styles["BodyText"]
        story = []
        for para in (text or "").split("\n\n"):
            safe = escape(para).replace("\n", "<br/>")
            story.append(Paragraph(safe if safe.strip() else "&nbsp;", body))
            story.append(Spacer(1, 8))
        doc.build(story)
        buf.seek(0)
        return buf.getvalue()

    def _rag_answer(self, k: int = 5):
        question = f"Create the company profile of {self.company_name}. USE ONLY the information from latest annual report."

        mode, hits = self._retrieve_hybrid_enhanced(question, k=k)
        ctx = self._build_context(hits)

        system_msg = self.profile_prompt
        user_msg = f"Question:\n{question}\n\nContext snippets (numbered):\n{ctx}"

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # 🧰 Tools (optional web search). Keep OFF by default to honor RAG purity.
        tools = [{"type": "web_search"}] if self.enable_web_search else []

        try:
            resp = self.web_openai.responses.create(
                model=self.model,
                input=messages,
                tools=[{"type": "web_search"}],
                # tool_choice='auto',
                # temperature=self.temperature,
                # top_p=self.top_p,
                max_output_tokens=self.max_output_tokens,
                reasoning={"effort": self.reasoning_effort},
                text={"verbosity": self.verbosity},
            )
            answer_text = resp.output_text

            # (Optional) Collect URL citations if web search was used
            if tools:
                urls = set()
                for item in getattr(resp, "output", []) or []:
                    if getattr(item, "type", "") == "message":
                        for c in getattr(item, "content", []) or []:
                            for ann in getattr(c, "annotations", []) or []:
                                if getattr(ann, "type", "") == "url_citation" and getattr(ann, "url", None):
                                    urls.add(ann.url)
                if urls:
                    answer_text += "\n\nSources (web):\n" + "\n".join(f"- {u}" for u in urls)

        except APIConnectionError as e:
            answer_text = f"Connection error calling OpenAI: {e}"

        return answer_text

agent1 = profileAgentWeb(
    'Radley + Co. Limited',
    k=200, max_text_recall_size=400, max_chars=10000,tool_choice='web_search',
    model='gpt-5', profile_prompt=finance_prompt_web
)

out_pdf = agent1._rag_answer()

k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


In [10]:
out_pdf

''

In [7]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model="gpt-5",
    tools=[{"type": "web_search"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

Here’s one: Today (September 11, 2025), volunteers across the U.S. marked the 9/11 Day of Service by packing meals for people in need. In New York, a two-day event that continued into today assembled more than two million meals aboard the USS Intrepid, part of coordinated projects in about 25 cities; organizers say tens of millions of Americans take part in acts of service nationwide. ([apnews.com](https://apnews.com/article/e8455f282943b3b6cb4f9a8765a16691))


In [19]:
import os, textwrap
import io

from typing import List, Dict, Optional
from xml.sax.saxutils import escape

from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError
from azure.search.documents.models import HybridSearch

from openai import AzureOpenAI, APIConnectionError
from prompts import new_system_finance_prompt, finance_prompt_web

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

load_dotenv(find_dotenv(), override=True)

# ---- Config (expects the same envs you already used) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

AOAI_ENDPOINT   = os.environ["AZURE_OPENAI_ENDPOINT"]            # https://<resource>.openai.azure.com
AOAI_API_VER    = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21")
AOAI_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]          # e.g., gpt-4o-mini / o3-mini / gpt-5 preview
AOAI_KEY        = os.getenv("AZURE_OPENAI_API_KEY")              # omit if using AAD

OPENAI_API_KEY  = os.getenv("FELIPE_OPENAI_API_KEY")        # required
OPENAI_MODEL    = os.getenv("FELIPE_OPENAI_MODEL", "gpt-5")  # e.g., "gpt-5" or "gpt-5-mini"


# ------------------ CODE

class profileAgent():

    """Hybrid (dense+sparse) RAG over Vector Store

    This Agent is responsible for creating Company Profiles. 
    It operates with gpt5.
    It is activated by a call on main rag when it is typed 'Create company profile'
    """

    def __init__(self, company_name, k, max_text_recall_size, max_chars, model, profile_prompt = finance_prompt_web):
        self.company_name = company_name

        self.k = k
        self.max_text_recall_size = max_text_recall_size
        self.model = model
        self.max_chars = max_chars

        self.azure_credentials = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
        self.search_client = SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=self.azure_credentials)

        self.az_openai = AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, api_key=AOAI_KEY, api_version=AOAI_API_VER)
        self.model = OpenAI(api_key=OPENAI_API_KEY)
        self.profile_prompt = profile_prompt

    def _retrieve_hybrid_enhanced(self, query: str, k: int = 10, max_text_recall_size:int = 200):
        sc = self.search_client
        try:
            vq = VectorizableTextQuery(text=query, k=k, fields=VECTOR_FIELD)
            # Prefer vector-only search (integrated vectorization). If your index isn't set up for it, this raises.
            results = sc.search(
                search_text=query, 
                vector_queries=[vq], 
                top=self.k, 
                query_type="semantic",
                query_caption="extractive", 
                hybrid_search=HybridSearch(max_text_recall_size=self.max_text_recall_size),
                query_caption_highlight_enabled=True,
                )
            mode = "hybrid + semantic"
        except HttpResponseError as e:
            # Fall back to lexical so you still get results while fixing vector config
            results = sc.search(search_text=query, top=k)
            mode = f"lexical (fallback due to: {e.__class__.__name__})"

        hits: List[Dict] = []
        for r in results:
            d = r.copy() if hasattr(r, "copy") else {k2: r[k2] for k2 in r}
            d["score"] = d.get("@search.reranker_score") or d.get("@search.score") or 0.0
            caps = d.get("@search.captions")
            if isinstance(caps, list) and caps:
                d["caption"] = getattr(caps[0], "text", None)
            hits.append(d)

        return mode, hits


    def _build_context(self, hits: List[Dict], text_field: str = TEXT_FIELD, max_chars: int = 20000) -> str:
        """Build a compact, numbered context block to feed the model."""
        lines = []
        total = 0
        for i, h in enumerate(hits, 1):
            title     = h.get("title")
            chunk_id  = h.get("chunk_id")
            snippet   = (h.get(text_field) or "")
            if not snippet:
                continue
            snippet = textwrap.shorten(snippet, width=700, placeholder=" ...")
            block = f"[{i}] title={title!r} | chunk_id={chunk_id} | score={h.get('score'):.4f}\n{snippet}"
            if total + len(block) > self.max_chars:
                break
            total += len(block)
            lines.append(block)
        return "\n\n---\n\n".join(lines)
    
    def _generate_pdf(self, text: str) -> bytes:

        buf = io.BytesIO()
        doc = SimpleDocTemplate(buf, pagesize=letter)
        styles = getSampleStyleSheet()
        body = styles["BodyText"]

        story = []
        # Treat double newlines as paragraph breaks; keep single newlines as <br/>
        for para in (text or "").split("\n\n"):
            safe = escape(para).replace("\n", "<br/>")
            story.append(Paragraph(safe if safe.strip() else "&nbsp;", body))
            story.append(Spacer(1, 8))

        doc.build(story)
        buf.seek(0)
        return buf.getvalue()

    def _rag_answer(self, k: int = 5, temperature: float = 0.2):
        question = f'Create the company profile of {self.company_name}. USE ONLY the information from the three latest annual report and use web_search.'

        mode, hits = self._retrieve_hybrid_enhanced(question, k=k)
        ctx = self._build_context(hits)

        system_msg = (
            self.profile_prompt
        )
        user_msg = f"Question:\n{question}\n\nContext snippets (numbered):\n{ctx}"

        client = self.model
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.
        
        # resp = client.chat.completions.create(
        #     model='gpt-5',
        #     messages=messages,
        # )
        stream = client.responses.stream(
            resp = client.responses.create(
                model="gpt-5",
                tools=[{"type": "web_search",
                        "search_context_size": "low"}],
                include=["web_search_call.action.sources"],
                input=messages
            )
        )
        stream.until_done()
        final = stream.get_final_response()
        answer_text = final.output_text
        
        # answer = resp.choices[0].message.content
        # mode_model = "non-streaming (fallback)"

        return answer_text
    
agent1 = profileAgent(
    'Radley + Co. Limited',
    k=50, max_text_recall_size=5000, max_chars=3000,
    model='gpt-5', profile_prompt=finance_prompt_web
)

out_pdf = agent1._rag_answer()
out_pdf

k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


InternalServerError: upstream connect error or disconnect/reset before headers. reset reason: connection termination

In [13]:
print(out_pdf)

Company: Radley + Co. Limited
Source for this profile: Radley + Co. Limited Annual Report for the 52‑week period ended 27 Apr‑24

1) Introduction Table (Company Snapshot)
- Primary Industry: Accessories (handbags and accessories) [2024 Annual Report, Strategic Report] [6]
- Incorporation Year: Not disclosed in FY24 annual report [2024 Annual Report] [2]
- Headquarters: Milton Keynes, United Kingdom [2024 Annual Report, Subsidiaries note lists the group’s registered office location used across entities] [10]
- Employees: Not disclosed in FY24 annual report sections available [2024 Annual Report, Staff costs note referenced but headcount figure not presented in available extract] [7]
- Operational KPIs:
  - High street stores operated: 2 [2024 Annual Report, Strategic Report] [6]
  - Outlet stores operated: 17 [2024 Annual Report, Strategic Report] [6]
  - Concessions operated: 36 [2024 Annual Report, Strategic Report] [6]

2) Business Overview (Bullets Only)
- The company operates multi

In [ ]:
from azure.blob_functions import companyHouseListAdd
from azure.adf_functions import trigger_function
from azure.search_functions import run_indexer

companyHouseListAdd(CompanyNumber = 'SC010528')
trigger_function(companyNumber='SC010528')
run_indexer()

/Users/felipesilverio/Documents/GitHub/azure_langproject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# app.py
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
import json
from rag import (
    retrieve_hybrid_enhanced,
    build_context,
    get_aoai_client
)
from typing import List, Dict, Optional
from gpts.gpt_assistants import question_to_machine, summarizer, general_assistant, maybe_route_to_action
from openai import OpenAI, APIConnectionError
import streamlit as st
from prompts import default_gpt_prompt
import time
load_dotenv(find_dotenv(), override=True)

# ---- Config (same Azure Search envs you already use) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

# ---- OpenAI (standard) config ----
OPENAI_API_KEY  = os.getenv("FELIPE_OPENAI_API_KEY")        # required
OPENAI_MODEL    = os.getenv("FELIPE_OPENAI_MODEL", "gpt-5")  # e.g., "gpt-5" or "gpt-5-mini"


class WebAgent():

    """
        - This class is responsible to operate calls and allow the usage of websearch
        - The websearch is activated through chat by mentioning "web search" in the paragraph
    """

    def __init__(self,
                k: int = 50,
                max_text_recall_size: int = 200,
                # max_chars: int,
                model: Optional[str] = OPENAI_MODEL,
                top = 20,
                max_output_tokens: int = 1200,
                reasoning_effort: str = "medium",      # "minimal" | "low" | "medium" | "high"
                verbosity: str = "medium",                 # "low" | "medium" | "high"
                tool_choice: str = "none",              # "none" | "auto" | {"type":"tool","name":"..."}
                streaming: bool = False
                ):

        # Parameters settings
        # self.company_name = company_name
        self.k = k
        self.max_text_recall_size = max_text_recall_size
        # self.max_chars = max_chars
        # ===================================
        # RAG PARAMETERS
        self.top = top
        self.k = k
        self.max_text_recall_size

        # ===================================
        # LLM settings
        self.model = model
        # self.temperature = temperature
        # self.top_p = top_p
        self.max_output_tokens = max_output_tokens
        self.reasoning_effort = reasoning_effort
        self.verbosity = verbosity
        self.streaming = streaming

        # OpenAI standard client
        self.web_openai = OpenAI(api_key=OPENAI_API_KEY)
        self.client = get_aoai_client()

    
    def _answer(self, question, stream = False):

        # 1. TRANSLATE TO MACHINE

        opt_user_query = question_to_machine(question, OPENAI_API_KEY)
        new_user_query = opt_user_query.output_text

        # 2. TOOL DETECTOR

        calls = maybe_route_to_action(new_user_query, self.client, self.model)

        if calls: #if no action detected, normal proccess
        # 3. PIPELINE CALL
            for call in calls:
                if call.function.name == "web_search":

                    try:
                        messages = [
                            {"role": "system", "content": default_gpt_prompt},
                            {"role": "user",   "content": new_user_query},
                        ]
                                
                        resp = self.web_openai.responses.create(
                            model=self.model,
                            input=messages,
                            tools=[{"type": "web_search"}],
                            tool_choice="auto",
                            # max_output_tokens=self.max_output_tokens,
                            reasoning={"effort": self.reasoning_effort},
                            text={"verbosity": self.verbosity},
                        )
                        return resp.output_text
                    except Exception as e:
                        return f'Activated web search but found error {e}'
                    
                elif call.function.name == "web_search_duo":
                    
                    # SPLITTING THE TEXT FOR EACH SOURCE
                    instruction = """
                    You are a financial assistant that receive instructions with multiple sources, and splits the request by source.
                    GIVE AS OUTPUT the parts of the question that DOESNT require the USE of WEB SEARCH.
                    """
                    offline_question = general_assistant(instruction, new_user_query, OPENAI_API_KEY, 'o3')
                    time.sleep(1.2)
                    instruction = """
                    You are a financial assistant that receive instructions with multiple sources, and splits the request by source.
                    GIVE AS OUTPUT the parts of the question that REQUIRES the use of WEB SEARCH.

                    Transform them into question optimized for web search
                    """
                    online_question = general_assistant(instruction, new_user_query, OPENAI_API_KEY, 'o3')
                    time.sleep(1.2)

                    # RETRIEVING THE ANSWERS
                    messages = [
                        {"role": "system", "content": default_gpt_prompt},
                        {"role": "user",   "content": online_question},
                    ]
                    resp_on = self.web_openai.responses.create(
                            model=self.model,
                            input=messages,
                            tools=[{"type": "web_search"}],
                            tool_choice="auto",
                            # max_output_tokens=self.max_output_tokens,
                            reasoning={"effort": self.reasoning_effort},
                            text={"verbosity": self.verbosity},
                        )
                    time.sleep(1.2)
                    resp_on_answer = resp_on.output_text
                
                    mode, hits = retrieve_hybrid_enhanced(query=offline_question, top = self.top, k = self.k, max_text_recall_size = self.max_text_recall_size)
                    ctx = build_context(hits)

                    user_msg = f"Question:\n{offline_question}\n\nContext snippets (numbered):\n{ctx}"

                    messages = [
                        {"role": "system", "content": default_gpt_prompt},
                        {"role": "user",   "content": user_msg},
                    ]
                    time.sleep(40)
                    resp_off = self.web_openai.responses.create(
                            model=self.model,
                            input=messages,
                            # max_output_tokens=self.max_output_tokens,
                            reasoning={"effort": "high"},
                            text={"verbosity": self.verbosity},
                            )
                    resp_off_answer = resp_off.output_text
                    print('Done resp off')
                    time.sleep(1.2)
                    summary = summarizer(resp_on_answer, resp_off_answer, OPENAI_API_KEY, self.model)

                    return summary
        
        
        # 2. RETRIEVE
        mode, hits = retrieve_hybrid_enhanced(query=new_user_query, top = self.top, k = self.k, max_text_recall_size = self.max_text_recall_size)
        ctx = build_context(hits)

        # 3. Call GPT
        system_msg = default_gpt_prompt

        user_msg = f"Question:\n{new_user_query}\n\nContext snippets (numbered):\n{ctx}"

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]
        

        resp = self.web_openai.responses.create(
                model=self.model,
                input=messages,
                # max_output_tokens=self.max_output_tokens,
                reasoning={"effort": "high"},
                text={"verbosity": self.verbosity},
            )
        answer_text = resp.output_text

        return answer_text

In [2]:
question = 'look for debt-related subjects Radley + Co. Limited in FY24 annual report and look for news related to Radley debt in the last month'

agent = WebAgent()
resp = agent._answer(question = question)

print(resp)

Translated to machine
Looked for actions
Web Search Duo PATH chosen
Separated questions


/Users/felipesilverio/Documents/GitHub/azure_langproject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


Finished resp on
Done RAG
Done resp off
Prompt One — 30-day UK scan (Aug–Sep 2025) on Radley + Co. Limited debt/financing

- No new UK media coverage or official notices in the last 30 days on Radley + Co. Limited regarding debt raises, refinancing, covenant issues, defaults, or insolvency; no relevant press releases on Radley’s website during the period. This is corroborated by the Companies House filing history (no new debt-related filings in the window), The Gazette (no notices in the period), and Radley’s Press & Media page (no debt/financing releases). (Companies House filing history: https://find-and-update.company-information.service.gov.uk/company/02573819/filing-history; The Gazette: https://www.thegazette.co.uk/notice/L-59768-1349843; Radley Press & Media: https://www.radley.co.uk/information/press-and-media)

- No new charges were created/delivered within the last 30 days; the most recent financing signals sit just outside the window (late Jul-25) with three new Companies Ho

In [65]:
from gpts.tools import TOOLS, TOOLS2, TOOLS3

def maybe_route_to_action(prompt, client, deployment) -> bool:
    """
    Returns True if the tool was invoked and handled here (so skip RAG),
    otherwise False to continue with your normal RAG flow.
    """
    sys_msg = """
        You are a router. 
        If the user asks to 'Create company profile (Name)', call the function with the extracted name. Otherwise, do nothing.
        If the user asks to 'Add company (Number)', call the function with the extracted number. Otherwise, do nothing.
        If the user asks for information with SOURCE or INFORMATION as 'web search', call the function 'web_search'.Otherwise, do nothing.
        If the user asks for information with SOURCE or INFORMATION as 'annual report + web_search', call the function 'web_search_duo'.Otherwise, do nothing.
    """

    try:
        resp = client.chat.completions.create(
            model=deployment,
            tools=TOOLS3,
            tool_choice="auto",
            messages=[
                {"role": "system",
                 "content": sys_msg},
                {"role": "user", "content": prompt},
            ],
        )
    except APIConnectionError:
        return False

    msg = resp.choices[0].message
    calls = getattr(msg, "tool_calls", None)

    return calls 

In [71]:
text = """
INFORMATION SOURCE: annnual report

Vector-search-optimized rewritten question:
Provide the latest news and updates from September 2025 (past 30 days) about Radley + Co. Limited, also known as Radley & Co Ltd, Radley and Co, and “Radley London,” the UK handbags and accessories brand. Focus on corporate announcements, press releases, product launches, financial results or funding, leadership/executive changes, store openings/closures, partnerships/collaborations, legal or Companies House filings, and any M&A or strategic initiatives. Return headline, date, source, and a one-sentence summary.

Web search query variants:
- "Radley & Co Ltd" news September 2025
- "Radley London" press release September 2025
- "Radley and Co Limited" latest news past month
- site:radley.co.uk (press OR "press release" OR news) September 2025
- site:find-and-update.company-information.service.gov.uk "Radley and Co Limited" filing history September 2025
- site:drapersonline.com "Radley London" September 2025
- site:retailgazette.co.uk Radley September 2025
- site:fashionnetwork.com "Radley London" September 2025
- "Radley London" partnership OR collaboration September 2025
- "Radley London" store opening OR closure September 2025
- "Radley London" appoints CEO OR CFO OR CMO September 2025
- "Radley London" investment OR funding OR acquisition September 2025
- "Radley London" marketing campaign OR ambassador September 2025
- "Radley London" lawsuit OR legal OR trademark September 2025
- "Radley London" results OR revenue OR profit September 2025
Tip: to reduce false matches (e.g., Radley College), add: -"Radley College" -school -Oxfordshire
"""

from rag import (
    get_aoai_client,
)
# from gpts.gpt_assistants import maybe_route_to_action

client = get_aoai_client()

calls = maybe_route_to_action(text, client, 'gpt-5')
for call in calls:
    print(call.function.name)

TypeError: 'NoneType' object is not iterable

In [68]:
for call in calls:
    print(call.function.name)

web_search


In [44]:
calls.choices[0].message.tool_calls[0]

ChatCompletionMessageFunctionToolCall(id='call_SqLu0gaY6NoZnp7rfVOrPtWg', function=Function(arguments='{"webSearch":"INFORMATION SOURCE: websearch\\n\\nVector-search-optimized rewritten question:\\nProvide the latest news and updates from September 2025 (past 30 days) about Radley + Co. Limited, also known as Radley & Co Ltd, Radley and Co, and “Radley London,” the UK handbags and accessories brand. Focus on corporate announcements, press releases, product launches, financial results or funding, leadership/executive changes, store openings/closures, partnerships/collaborations, legal or Companies House filings, and any M&A or strategic initiatives. Return headline, date, source, and a one-sentence summary.\\n\\nWeb search query variants:\\n- \\"Radley & Co Ltd\\" news September 2025\\n- \\"Radley London\\" press release September 2025\\n- \\"Radley and Co Limited\\" latest news past month\\n- site:radley.co.uk (press OR \\"press release\\" OR news) September 2025\\n- site:find-and-upda

In [ ]:
ax = calls.choices[0].message
ax

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_UZ75FUm0GnEMdWPTt3XG43FR', function=Function(arguments='{"webSearch":"INFORMATION SOURCE: websearch"}', name='web_search'), type='function')])

In [63]:
ax2

[ChatCompletionMessageFunctionToolCall(id='call_UZ75FUm0GnEMdWPTt3XG43FR', function=Function(arguments='{"webSearch":"INFORMATION SOURCE: websearch"}', name='web_search'), type='function')]

In [62]:
ax2 = getattr(ax, "tool_calls", None)
for i in ax2:
    print(i.function.name)

web_search
